# Information Architectures Final Project

### Submitted by :

* Muhammad Bilal Jamil
* Te Li
* Mark Kaplan

### Submitted to : Prof. Chiazza

In [9]:
#Importing useful libraries to work with APIs:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import s3fs as s3
import logging
import boto3
from botocore.exceptions import ClientError
import awscli
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import time
import io
import os

In [11]:
# loading our final project 1st 2 data sources:
source1_df = pd.read_csv(r'/Users/Bilal/Documents/Bilal Files/Yeshiva/Spring 2022/Final Project/Datasets/Traffic_Tickets_Issued__Four_Year_Window.csv')
source2_df = pd.read_csv(r'/Users/Bilal/Documents/Bilal Files/Yeshiva/Spring 2022/Final Project/Datasets/Motor_Vehicle_Crashes_-_Violation_Information__Three_Year_Window.csv')

In [12]:
source1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11902573 entries, 0 to 11902572
Data columns (total 11 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Violation Charged Code  object 
 1   Violation Description   object 
 2   Violation Year          int64  
 3   Violation Month         int64  
 4   Violation Day of Week   object 
 5   Age at Violation        float64
 6   Gender                  object 
 7   State of License        object 
 8   Police Agency           object 
 9   Court                   object 
 10  Source                  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 998.9+ MB


In [19]:
source1_df.head()

,Violation Charged Code,Violation Description,Violation Year,Violation Month,Violation Day of Week,Age at Violation,Gender,State of License,Police Agency,Court,Source
0,1229C3,NO SEAT BELT ADULT,2018,1,SUNDAY,49.0,M,NEW YORK,NYC POLICE DEPT,BRONX TVB,TVB
1,1180D12,SPEED IN ZONE 11-30,2018,1,SUNDAY,17.0,M,NEW YORK,NYC POLICE DEPT,BRONX TVB,TVB
2,37512A2,SIDEWINGS/SIDEWINDOWS/NON/TRANSPARENT,2018,1,SUNDAY,54.0,M,NEW JERSEY,NYC POLICE DEPT,MANHATTAN SOUTH TVB,TVB
3,37512A3,REAR SIDE WINDOWS NON/TRANSPARENT,2018,1,SUNDAY,21.0,M,NEW YORK,NYC POLICE DEPT,MANHATTAN SOUTH TVB,TVB
4,1180D12,SPEED IN ZONE 11-30,2018,1,SUNDAY,22.0,M,NEW YORK,NYC POLICE DEPT,BROOKLYN SOUTH TVB,TVB


In [13]:
source2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238044 entries, 0 to 238043
Data columns (total 4 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Year                   238044 non-null  int64 
 1   Violation Description  238044 non-null  object
 2   Violation Code         238044 non-null  object
 3   Case Individual ID     238044 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 7.3+ MB


In [2]:
# Creating a functions which will help me create the S3 bucket in my AWS S3 environment:

def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [3]:
create_bucket('final-project-ia-2022')

True

In [4]:
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  dealerships.nys
  final-project-ia-2022
  information-arch-mbj-assignment-8a
  m10-assignment-charities-bucket
